In [1]:
from transformers import pipeline

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

In [2]:
model = AutoModelForCausalLM.from_pretrained('EleutherAI/pythia-12b', cache_dir = './hf-models')

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
tokenizer = AutoTokenizer.from_pretrained(
    'EleutherAI/pythia-12b', 
    cache_dir = './hf-models',
    additional_special_tokens = (
        '<|query|>',
        '</query/>',
        '<|response|>',
        '</response/>',
        '<|result|>',
        '</result/>',
        '<|summary|>',
        '</summary/>',
        '<|results|>',
        '</results/>',
        '<|result-sentence|>'
    )
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
import jsonlines

In [5]:
with open("../kcc_data/jsonlines/data.jsonl", 'r') as reader:
    data = [i[1:-2] for i in reader]

In [6]:
import torch

In [9]:
from oslo import ParallelContext, ParallelMode
from oslo.torch.nn.parallel import TensorParallel

tp_size = 2
tp_depth = 1

parallel_context = ParallelContext.from_torch(
    data_parallel_size=1,
    pipeline_parallel_size=1,
    tensor_parallel_size=tp_size,
    tensor_parallel_mode=ParallelMode.TENSOR_1D,
)
model = TensorParallel(model, parallel_context)
oslo.ready(model, parallel_context)

KeyError: 'RANK'

In [9]:
optimizer = torch.optim.Adam(model.parameters(), lr = 1e-6)

In [ ]:
from tqdm.auto import tqdm

In [ ]:
bar = tqdm(data)

In [24]:
for record in data:
    tokens = tokenizer(record, return_tensors = 'pt')
    
    optimizer.zero_grad()
    loss = model.forward(**tokens)
    loss.backward()
    optimizer.step()
    

KeyboardInterrupt: 